In [ ]:
#OLD CODE HERE FOR THE TIME BEING
# Code to check for clusters that span at least two bands
def cluster_cleaning(cluster_df, topic_df):
    new_rows = []
    bad_rows = []
    for clst in cluster_df.index:
        # HDBSCAN labels noise as -1 so we skip this row for cleaning
        if clst != -1:
            if cluster_df.loc[clst].band_min != cluster_df.loc[clst].band_max:
                olap_band_min = cluster_df.loc[clst].band_min.astype('int64')
                olap_band_max = cluster_df.loc[clst].band_max.astype('int64')
                olap_band_mode = cluster_df.loc[clst].band_mode.astype('int64')
                olap_min_freq = cluster_df.loc[clst].min_freq
                olap_max_freq = cluster_df.loc[clst].max_freq

                # Check to see that the cluster doesn't span more than two bands
                # If it does to this the cluster is far too large and clusters need to be tuned better
                if olap_band_max - olap_band_min > 1:
                    #raise ValueError('Cluster spans more than 2 bands. Re-parameterize clusters.')
                    print("WARNING: Generated cluster spans more than two bands.")
                
                # Otherwise, we split the cluster into two different clusters on the band boundaries
                new_row1_min_freq = olap_min_freq
                new_row1_max_freq = band_cutoffs[olap_band_max-1]
                new_row2_min_freq = band_cutoffs[olap_band_max-1]
                new_row2_max_freq = olap_max_freq

                # There are cases where the band transition in the data is wrong
                # Measurements in band 4 are misclassified to band 5 it seems if they are very close to the boundary between bands
                # If this is the case, we don't make new rows and simply set the band mode for the 'bad' row to the band with the higher mode
                if (new_row1_min_freq > new_row1_max_freq) or (new_row2_min_freq > new_row2_max_freq):
                    break

                reassign_measures = topic_df[topic_df.cluster_label == clst].med_freq\
                    .sort_values()\
                    .to_list()
                new_row1_measures = []
                new_row1_projects = []
                new_row2_measures = []
                new_row2_projects = []

                # Loop over reassign_measures and build lists for each new cluster
                for meas in reassign_measures:
                    if meas <= new_row2_min_freq:
                        new_row1_measures.append(meas)
                    else:
                        new_row2_measures.append(meas)

                # Generate column values for new rows (clusters)
                new_row1_count = len(new_row1_measures)
                new_row2_count = len(new_row2_measures)
                new_row1_mean = np.mean(new_row1_measures)
                new_row2_mean = np.mean(new_row2_measures)
                new_row1_band_min = olap_band_min
                new_row1_band_max = olap_band_min
                new_row2_band_min = olap_band_max
                new_row2_band_max = olap_band_max
                new_row1_band_mode = olap_band_mode.astype('int64')
                new_row2_band_mode = olap_band_mode.astype('int64')

                # Make new row lists to add to data frame
                new_row1 = [new_row1_mean,
                            new_row1_min_freq,
                            new_row1_max_freq,
                            new_row1_count,
                            new_row1_band_min,
                            new_row1_band_max,
                            new_row1_band_mode
                            ]
                
                new_row2 = [new_row2_mean,
                            new_row2_min_freq,
                            new_row2_max_freq,
                            new_row2_count,
                            new_row2_band_min,
                            new_row2_band_max,
                            new_row2_band_mode
                            ]
                
                # Add new rows (clusters) to list to ultimately alter cluster_df
                # We don't want to alter the data frame we're looping over in the loop
                new_rows.append(new_row1)
                cluster_df.loc[len(cluster_df.index)] = new_row1
                cluster_df.loc[len(cluster_df.index)] = new_row2
                bad_rows.append(clst)

    # Drop and add affected rows
    if len(bad_rows) != 0:
        for br in range(len(bad_rows)):
            cluster_df = cluster_df.drop(bad_rows[br], axis=0)
            
    # Set axis to maintain HDBSCAN noise is labeled -1
    cluster_df.index = (list(range(-1,cluster_df.shape[0]-1)))
    return cluster_df